# Step 1: Merge data into a single covariate tensor

In [61]:
import pickle

# %load_ext autoreload
# %autoreload 0

In [65]:
len(cotensor)

64776522

In [ ]:
# merge into a large tensor, index by word, word, topic, time

subs = ["Conservative", "NeutralPolitics", "PoliticalDiscussion", "SandersForPresident", "The_Donald", "politics"]

cotensor = {}

for sub in subs:
    print sub
    for i in xrange(72):
        print i
        sub_cotensor = pickle.load(open("../../Data/" + sub + "/" + str(i) + ".p", 'rb'))
        for key in sub_cotensor:
            a, b, c = key
            if (a, b, c, i / 6) in cotensor:
                cotensor[(a, b, c, i / 6)] += sub_cotensor[key]
            else:
                cotensor[(a, b, c, i / 6)] = sub_cotensor[key]

Conservative
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
NeutralPolitics
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
PoliticalDiscussion
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
SandersForPresident
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
The_Donald
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
3

In [66]:
# unique words
wordlist = {}
for key in cotensor:
    word1, word2, sub, time = key
    count = cotensor[key]
    if word1 in wordlist:
        wordlist[word1] += count
    else:
        wordlist[word1] = count
    if word2 in wordlist:
        wordlist[word2] += count
    else:
        wordlist[word2] = count
        
# sort words
words = wordlist.keys()
counts = [wordlist[word] for word in words]
counts, words = zip(*sorted(zip(counts, words)))

In [ ]:
# just take the top 15k most common words
# TODO: filter out silly words? i.e. words that are just numbers, punctuation marks

words_15k, counts_15k = words[-15000:], counts[-15000:]
word_count_15k = {words_15k[i] : counts_15k[i] for i in xrange(len(words_15k))}
word_dict_15k_redone = {word : i for (i, word) in enumerate(word_count_15k)}
pickle.dump(word_dict_15k_redone, open("word_dict_politics.p", 'wb'))

In [115]:
# probably helpful?

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

40

In [ ]:
sub_dict = {"Conservative" : 0, "NeutralPolitics" : 1, "PoliticalDiscussion" : 2, "SandersForPresident" : 3, "The_Donald" : 4, "politics" : 5}
cotensor_15k = {}
for key in cotensor:
    word1, word2, sub, time = key
    if word1 in word_dict_15k_redone and word2 in word_dict_15k_redone:
        cotensor_15k[(word_dict_15k_redone[word1], word_dict_15k_redone[word2], sub_dict[sub], time)] = cotensor[key]

In [ ]:
# cotensor is too big. free up some space

%xdel cotensor

# PCA example

In [ ]:
# ignore this for now - this is just to compare against a PCA baseline

matrix_15k = {}
for key in cotensor:
    word1, word2, sub, time = key
    if (word1, word2) in matrix_15k:
        matrix_15k[(word1, word2)] += cotensor[key]
    else:
        matrix_15k[(word1, word2)] = cotensor[key]

In [14]:
%xdel cotensor

# Train embeddings

In [ ]:
# play with hyperparameters here!

import tf_glove

In [15]:
model = tf_glove.GloVeModel(embedding_size=200, cooccurrence_cap=4000, scaling_factor=0.75, learning_rate=0.10, cooccur=cotensor_15k)

In [ ]:
model.fit_to_corpus()
model.train(num_epochs=200)
model.flush_embeddings()

# Slices

In [20]:
# this is if you want to do specific things with 

import numpy as np

sub_counts = np.zeros(6)
time_counts = np.zeros(12)
for key in cotensor_15k:
    word1, word2, sub, time = key
    sub_counts[sub] += cotensor_15k[key]
    time_counts[time] += cotensor_15k[key]

In [21]:
sub_counts

array([ 11752552.,   4609578.,  42645320.,  31743198.,  37620670.,
        51170326.])

In [22]:
time_counts

array([ 20413132.,  20683720.,  18301272.,  12704992.,  11432898.,
        12053328.,   9540776.,  12934182.,  14434556.,  18086192.,
        12227210.,  16729386.])